In [9]:
import pandas as pd
import os
from datetime import datetime as dt
from sqlalchemy import create_engine

In [10]:
# ufo_data_path = "Resources/nuforc_reports.csv"
ufo_data_path = "Resources/nuforc_reports.csv"
ufo_kaggle_path = "Resources/ufo_sighting_data.csv"

In [15]:
# Import the ufo data from Data.world and drop N/A's
nuforc_df = pd.read_csv(ufo_data_path)
nuforc_df.dropna(axis = 0, how = "any", inplace=True)

# Define which columns to keep in the dataframe
nuforc_df = nuforc_df[['date_time', 'city', 'state', 'shape', 'duration', 'summary', 'city_latitude', 'city_longitude']]

# Rename column headings
nuforc_df.rename(columns={'date_time' : 'date_time',
                              'city' : 'city',
                              'state' : 'state',
                              'shape' : 'ufo_shape',
                              'duration' : 'encounter_duration',
                              'summary' : 'encounter_summary',
                              'city_latitude' : 'latitude',
                              'city_longitude' : 'longitude'
                             }, inplace=True)

# Format fields
nuforc_df['state'] = nuforc_df['state'].str.upper()
nuforc_df['city'] = nuforc_df['city'].str.title()

nuforc_df.head()

,date_time,city,state,ufo_shape,encounter_duration,encounter_summary,latitude,longitude
0,2019-12-12T18:43:00,Chester,VA,light,5 seconds,My wife was driving southeast on a fairly popu...,37.343152,-77.408582
1,2019-03-22T18:30:00,Rocky Hill,CT,circle,3-5 seconds,I think that I may caught a UFO on the NBC Nig...,41.664800,-72.639300
3,2019-04-17T02:00:00,Ottawa,ON,teardrop,10 seconds,I was driving towards the intersection of fall...,45.381383,-75.708501
5,2019-04-02T20:25:00,Kirbyville,TX,disk,15 minutes,"The object has flashing lights that are green,...",30.677200,-94.005200
6,2019-05-01T11:00:00,Tucson,AZ,unknown,5 minutes,Description is the same as Washington DC event...,32.259941,-110.927542


In [4]:
# Import the ufo data from Kaggle.com and drop N/A's
kaggle_ufo_df = pd.read_csv(ufo_kaggle_path, low_memory=False)
kaggle_ufo_df.dropna(axis=0, how="any", inplace=True)


# Define which columns to keep in the dataframe
columns=['Date_time', 'city', 'state/province', 'UFO_shape', 'described_duration_of_encounter', \
         'description', 'latitude', 'longitude']

kaggle_ufo_df = kaggle_ufo_df[columns]

# Rename column headings
kaggle_ufo_df.rename(columns={'Date_time' : 'date_time',
                              'city' : 'city',
                              'state/province' : 'state',
                              'UFO_shape' : 'ufo_shape',
                              'described_duration_of_encounter' : 'encounter_duration',
                              'description' : 'encounter_summary',
                              'latitude' : 'latitude',
                              'longitude' : 'longitude'
                             }, inplace=True)

# Format fields 
kaggle_ufo_df['state'] = kaggle_ufo_df['state'].str.upper()
kaggle_ufo_df['city'] = kaggle_ufo_df['city'].str.title()

kaggle_ufo_df.head()

,date_time,city,state,ufo_shape,encounter_duration,encounter_summary,latitude,longitude
0,10/10/1949 20:30,San Marcos,TX,cylinder,45 minutes,This event took place in early fall around 194...,29.8830556,-97.941111
3,10/10/1956 21:00,Edna,TX,circle,1/2 hour,My older brother and twin sister were leaving ...,28.9783333,-96.645833
4,10/10/1960 20:00,Kaneohe,HI,light,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,21.4180556,-157.803611
5,10/10/1961 19:00,Bristol,TN,sphere,5 minutes,My father is now 89 my brother 52 the girl wit...,36.595,-82.188889
7,10/10/1965 23:45,Norwalk,CT,disk,20 minutes,A bright orange color changing to reddish colo...,41.1175,-73.408333


In [5]:
# Join the two dataframes together into one
ufo_df = pd.concat(objs = [nuforc_df, kaggle_ufo_df], ignore_index = True)
ufo_df.head()

,date_time,city,state,ufo_shape,encounter_duration,encounter_summary,latitude,longitude
0,2019-12-12T18:43:00,Chester,VA,light,5 seconds,My wife was driving southeast on a fairly popu...,37.3432,-77.408582
1,2019-03-22T18:30:00,Rocky Hill,CT,circle,3-5 seconds,I think that I may caught a UFO on the NBC Nig...,41.6648,-72.639300
2,2019-04-17T02:00:00,Ottawa,ON,teardrop,10 seconds,I was driving towards the intersection of fall...,45.3814,-75.708501
3,2019-04-02T20:25:00,Kirbyville,TX,disk,15 minutes,"The object has flashing lights that are green,...",30.6772,-94.005200
4,2019-05-01T11:00:00,Tucson,AZ,unknown,5 minutes,Description is the same as Washington DC event...,32.2599,-110.927542


### Connect to PostgreSQL database

In [ ]:
rds_connection_string = "postgres:password@localhost:5432/ufo_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [1]:
# Check to see if the table name exists
engine.table_names()

NameError: name 'engine' is not defined

### Export the DataFrame to SQL

In [ ]:
ufo_df.to_sql(name='ufo_data', con=engine, if_exists='replace', index=False)

### Verify that the data can be read into a DataFrame from SQL

In [ ]:
pd.read_sql_query('select * from ufo_data', con=engine).head()